# Check Retention Cohort Table
* StelllarAlgo Data Science
* Ryan Kazmerik
* Oct 21, 2022

In [2]:
import boto3
import pandas as pd

from shared_utilities import helpers

In [3]:
session = helpers.get_aws_session("Stellaralgo-DataScienceAdmin")

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


In [4]:
CLUSTER = "prod-app"
DATABASE = "stlrcanucks"
LKUPCLIENTID = 7

CNXN = helpers.get_redshift_connection(CLUSTER, DATABASE)

Authorized as AROASQ4JELIXYLYV6P4UV:rkazmerik@stellaralgo.com


In [6]:
sql= f"""
    SELECT * FROM {DATABASE}.dw.cohortproductpropensityscores
    WHERE lkupclientid = {LKUPCLIENTID}
    AND date_effective_end IS NULL
"""

try:
    cursor = CNXN.cursor()
    cursor.execute(sql)
    data = cursor.fetchall()

    cols = [row[0] for row in cursor.description]
    df_scores = pd.DataFrame(data=data, columns=cols)

    print(f"ROWS RETRIEVED: {df_scores.shape[0]}")
    print(f"MAX SCORE DATE: {df_scores['scoredate'].max()}")

except Exception as e:
    print(f"ERROR:, {e}")
    cursor.execute("ROLLBACK")

finally:
    cursor.close()


ROWS RETRIEVED: 7474
MAX SCORE DATE: 2022-10-19 12:00:55


In [7]:
df_scores.groupby(["product","product_current"]).agg({
    "product": "count",
})

product
product     product_current         
Any         Full Season            6
            Individual            20
            Mini Pack              5
            None                5077
            Quarter                1
Full Season None                 828
Half Season Half Season            1
            None                 768
Quarter     None                 768

### Done